In [13]:
import pandas as pd
from tqdm.notebook import tqdm

tqdm.pandas()
pd.set_option('display.max_colwidth', None)

In [2]:
SCORE_CARDS = False

In [3]:
data = pd.read_csv("datasets/twitter_sentiment_data.csv")
data.sample()

,sentiment,message,tweetid,roberta_preprocessed,roberta_pred,roberta_proba
20807,2,RT @EcoInternet3: Five African states to benefit from $1 billion AfDB #climate change kitty: Asoko Insight https://t.co/R878EzuFwU #environ…,861359060448407552,RT @EcoInternet3: Five African states to benefit from $1 billion AfDB #climate change kitty: Asoko Insight https://t.co/R878EzuFwU #environ...,0_0,0.999801


- 2 (News): the tweet links to factual news about climate change
- 1 (Pro): the tweet supports the belief of man-made climate change
- 0 (Neutral): the tweet neither supports nor refutes the belief of man-made climate change
- -1 (Anti): the tweet does not believe in man-made climate change

In [4]:
data.sentiment.value_counts().to_frame()

,sentiment
1,22962
2,9276
0,7715
-1,3990


In [5]:
if SCORE_CARDS:
    import pickle
    from scipy.special import softmax
    from utils import roberta_model, preprocess_text

    data['roberta_preprocessed'] = preprocess_text(data["message"])

    predictions, raw_outputs = roberta_model.predict(list(data.roberta_preprocessed))

    with open('cards/models/label_encoder.pkl', 'rb') as f:
        le = pickle.load(f)

    data['roberta_pred'] = le.inverse_transform(predictions)
    data['roberta_proba'] = [max(softmax(element[0])) for element in raw_outputs]

    data.to_csv("datasets/twitter_sentiment_data.csv", index=False)

In [9]:
df_confusion = pd.crosstab(data["sentiment"], data["roberta_pred"])
df_confusion["total"] = df_confusion.sum(axis=1)
df_confusion

roberta_pred,0_0,1_1,1_2,1_3,1_4,1_6,1_7,2_1,2_3,3_1,3_2,3_3,4_1,4_2,4_4,4_5,5_1,5_2,total
sentiment,,,,,,,,,,,,,,,,,,,
-1,2478,13,12,60,32,3,26,108,32,0,4,0,56,6,0,0,61,1099,3990
0,7403,3,4,24,8,0,9,13,4,1,10,0,14,11,3,0,13,195,7715
1,21605,6,3,44,17,0,10,27,60,3,22,4,94,27,8,3,51,978,22962
2,8949,2,3,3,23,3,6,10,37,2,27,0,36,2,0,0,30,143,9276


In [21]:
data[(data.sentiment==-1)&(data.roberta_pred=="0_0")].sample(5)

,sentiment,message,tweetid,roberta_preprocessed,roberta_pred,roberta_proba
9411,-1,"Tree-huggin' hippie! Man walking BAREFOOT across America to protest climate change, 'save the earth'… https://t.co/TS5f2lfkEY",809193291736612867,"Tree-huggin' hippie! Man walking BAREFOOT across America to protest climate change, 'save the earth'... https://t.co/TS5f2lfkEY",0_0,0.999711
42302,-1,Excellent appearance on @megynkelly once again #Obama is huddling with foreign leaders about...climate change https://t.co/aOhg8kxZw8,748368819303890944,Excellent appearance on @megynkelly once again #Obama is huddling with foreign leaders about...climate change https://t.co/aOhg8kxZw8,0_0,0.999462
19740,-1,@TwitterMoments Too bad there is no evidence for global warming.,855863239124680704,@TwitterMoments Too bad there is no evidence for global warming.,0_0,0.996318
38206,-1,$q$And that$q$s why I don$q$t believe in global warming. We didn$q$t have cars and we made it through the the ice age obviously!$q$\n-Lisa Scharborough,643890365729239040,$q$And that$q$s why I don$q$t believe in global warming. We didn$q$t have cars and we made it through the the ice age obviously!$q$\n-Lisa Scharborough,0_0,0.998534
27468,-1,RT @CharlieDaniels: Wonder if Hillary will blame her loss on global warming\nShes used up about everything else,912513008290734080,RT @CharlieDaniels: Wonder if Hillary will blame her loss on global warming\nShes used up about everything else,0_0,0.999757
